# Scraping mine data with `.apply`

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Doing this programmatically

## First, scraping a single page

### Import your imports

In [93]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
driver = webdriver.Chrome()


### Searching for a mine

Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm) and use Selenium to search for `3503598`

- *TIP: You might need to use the Selenium code to scroll down to the right spot on the page. Or not!*
- *TIP: Use `.send_keys` to type into the box*
- *TIP: On pages that never change, you can usually just use XPath if you're feeling lazy*

In [17]:
driver.get("https://arlweb.msha.gov/drs/drshome.htm")


In [18]:
text_input = driver.find_element_by_name('MineId')

In [19]:
driver.execute_script("arguments[0].scrollIntoView(true)", text_input)

In [20]:
text_input.send_keys('3503598')

In [21]:
button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
button.click()

### Finding reports

On the "Report Selection Page" (where you should be after you search), use Selenium to...

- Change the **Beginning Date** to `1/1/1995`
- Select the report type of `Violations`
- Click **Get Report**

.

- *TIP: Remember, if someone isn't on the page Selenium can't click it!*

In [22]:
beginning_date = driver.find_element_by_name('BDate')
driver.execute_script("arguments[0].scrollIntoView(true)", beginning_date)
beginning_date.send_keys('1/1/1995')

In [23]:
select_button = driver.find_elements_by_name('Submit')[3]
select_button.send_keys('yes')
select_button.click()

In [24]:
report_button = driver.find_element_by_name('submit')
report_button.click()

### Saving reports

Save all of the rows of data on that page into a new dataframe. Each column is its own column, **and you also need to save the URL under the 'Standard' column.** Here, I even made you a blank dictionary:

```python
data = {}
data['violator'] = ''
data['contract_id'] = ''
data['citation_no'] = ''
data['case_no'] = ''
data['date_issues'] = ''
data['final_order_date'] = ''
data['section_of_act'] = ''
data['date_terminated'] = ''
data['citation'] = ''
data['s_and_s'] = ''
data['standard'] = ''
data['standard_url'] = ''
data['proposed_penalty'] = ''
data['citation_status'] = ''
data['current_penalty'] = ''
data['amount_paid'] = ''
```

- *TIP: Some of those table rows aren't what you want. How can you tell them apart from the good ones? (the previous mine owner ones are okay, I just mean the weird headers)*
- *TIP: I sense `.find_elements` + a lot of square brackets*
- *TIP: This is just like scraping a search results page!*
- *TIP: For the URL, you'll need to find the `a` inside of the cell*
- *TIP: class name is sadly not going to save your life here, because some of the `tr`s and `td`s have the same class! It's stupid. But there's a trick: CSS selectors! Something like `div#container` finds a `div` with the id of `container`, while `span.important` finds a `span` with the class of `important`. It should be helpful! And use `.find_elements_by_` + tab to see what the command is*

In [25]:
violator = driver.find_elements_by_class_name('drsviols')[17].text
violator = driver.find_elements_by_css_selector('td.drsviols')[0].text

violator

'Newberg Rock & Dirt'

In [26]:
contract_id = driver.find_elements_by_class_name('drsviols')[18].text
contract_id = driver.find_elements_by_css_selector('td.drsviols')[1].text
contract_id

'8992368  '

In [27]:
citation_no = driver.find_elements_by_class_name('drsviols')[19].text
citation_no = driver.find_elements_by_css_selector('td.drsviols')[1].text
citation_no

'8992368  '

In [28]:
case_no = driver.find_elements_by_css_selector('td.drsviols')[2].text


date_issues = driver.find_elements_by_css_selector('td.drsviols')[3].text
date_issues
final_order_date = driver.find_elements_by_css_selector('td.drsviols')[4].text
section_of_act = driver.find_elements_by_css_selector('td.drsviols')[5].text
date_terminated = driver.find_elements_by_css_selector('td.drsviols')[6].text
citation = driver.find_elements_by_css_selector('td.drsviols')[7].text
s_and_s = driver.find_elements_by_css_selector('td.drsviols')[8].text
standard = driver.find_elements_by_css_selector('td.drsviols')[9].text

standard2 = driver.find_elements_by_css_selector('td.drsviols')[9].find_element_by_tag_name('a')
standard_url = standard2.get_attribute('href')

proposed_penalty = driver.find_elements_by_css_selector('td.drsviols')[10].text
citation_status = driver.find_elements_by_css_selector('td.drsviols')[11].text
current_penalty = driver.find_elements_by_css_selector('td.drsviols')[12].text
amount_paid = driver.find_elements_by_css_selector('td.drsviols')[13].text
amount_paid

'116.00 '

In [29]:
reports = driver.find_elements_by_css_selector('tr.drsviols')

my_report = []

for report in reports:
    data = {}
    data['violator'] = report.find_elements_by_css_selector('td.drsviols')[0].text
    data['contract_id'] = " "
    data['citation_no'] = report.find_elements_by_css_selector('td.drsviols')[1].text
    data['case_no'] = report.find_elements_by_css_selector('td.drsviols')[2].text
    data['date_issues'] = report.find_elements_by_css_selector('td.drsviols')[3].text
    data['final_order_date'] = report.find_elements_by_css_selector('td.drsviols')[4].text
    data['section_of_act'] = report.find_elements_by_css_selector('td.drsviols')[5].text
    data['date_terminated'] = report.find_elements_by_css_selector('td.drsviols')[6].text
    data['citation'] = report.find_elements_by_css_selector('td.drsviols')[7].text
    data['s_and_s'] = report.find_elements_by_css_selector('td.drsviols')[8].text
    data['standard'] = report.find_elements_by_css_selector('td.drsviols')[9].text
    
    standard2 = driver.find_elements_by_css_selector('td.drsviols')[9].find_element_by_tag_name('a')
    data['standard_url'] = standard2.get_attribute('href')
    data['proposed_penalty'] = report.find_elements_by_css_selector('td.drsviols')[10].text
    data['citation_status'] = report.find_elements_by_css_selector('td.drsviols')[11].text
    data['current_penalty'] = report.find_elements_by_css_selector('td.drsviols')[12].text
    data['amount_paid'] = report.find_elements_by_css_selector('td.drsviols')[13].text
    my_report.append(data)
    

my_report

[{'violator': 'Newberg Rock & Dirt',
  'contract_id': ' ',
  'citation_no': '8992368  ',
  'case_no': '000440809',
  'date_issues': '5/2/2017',
  'final_order_date': '7/23/2017 ',
  'section_of_act': '104(a)',
  'date_terminated': '5/2/2017',
  'citation': 'C',
  's_and_s': 'N',
  'standard': '56.14107(a)',
  'standard_url': 'http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf',
  'proposed_penalty': '116.00',
  'citation_status': 'Closed',
  'current_penalty': '116.00 ',
  'amount_paid': '116.00 '},
 {'violator': 'Newberg Rock & Dirt',
  'contract_id': ' ',
  'citation_no': '8992214  ',
  'case_no': '000435717',
  'date_issues': '2/7/2017',
  'final_order_date': '5/21/2017 ',
  'section_of_act': '104(a)',
  'date_terminated': '2/8/2017',
  'citation': 'C',
  's_and_s': 'N',
  'standard': '56.14107(a)',
  'standard_url': 'http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf',
  'proposed_penalty': '116.00'

### Saving that data

Save the dataframe to a CSV file called `3503598-violations.csv` (that's the TDLR code)

In [30]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
df = pd.DataFrame(my_report)
df.head()

,amount_paid,case_no,citation,citation_no,citation_status,contract_id,current_penalty,date_issues,date_terminated,final_order_date,proposed_penalty,s_and_s,section_of_act,standard,standard_url,violator
0,116.00,000440809,C,8992368,Closed,,116.00,5/2/2017,5/2/2017,7/23/2017,116.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,Newberg Rock & Dirt
1,116.00,000435717,C,8992214,Closed,,116.00,2/7/2017,2/8/2017,5/21/2017,116.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,Newberg Rock & Dirt
2,100.00,000398910,C,8872461,Closed,,100.00,10/21/2015,10/27/2015,1/13/2016,100.00,N,104(a),56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,Newberg Rock & Dirt
3,100.00,000398910,C,8872462,Closed,,100.00,10/21/2015,10/27/2015,1/13/2016,100.00,N,104(a),56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,Newberg Rock & Dirt
4,100.00,000383840,C,8790965,Closed,,100.00,4/15/2015,4/15/2015,7/17/2015,100.00,N,104(a),56.13015,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,Newberg Rock & Dirt


In [31]:
df = df[['violator', 'contract_id', 'citation_no', 'case_no', 'date_issues', 'final_order_date', 'section_of_act', 'date_terminated', 'citation', 's_and_s', 'standard', 'standard_url', 'proposed_penalty', 'citation_status', 'current_penalty', 'amount_paid']]
df

,violator,contract_id,citation_no,case_no,date_issues,final_order_date,section_of_act,date_terminated,citation,s_and_s,standard,standard_url,proposed_penalty,citation_status,current_penalty,amount_paid
0,Newberg Rock & Dirt,,8992368,000440809,5/2/2017,7/23/2017,104(a),5/2/2017,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,116.00,Closed,116.00,116.00
1,Newberg Rock & Dirt,,8992214,000435717,2/7/2017,5/21/2017,104(a),2/8/2017,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,116.00,Closed,116.00,116.00
2,Newberg Rock & Dirt,,8872461,000398910,10/21/2015,1/13/2016,104(a),10/27/2015,C,N,56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
3,Newberg Rock & Dirt,,8872462,000398910,10/21/2015,1/13/2016,104(a),10/27/2015,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
4,Newberg Rock & Dirt,,8790965,000383840,4/15/2015,7/17/2015,104(a),4/15/2015,C,N,56.13015,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
5,Newberg Rock & Dirt,,8790964,000383840,4/13/2015,7/17/2015,104(a),4/15/2015,C,N,56.4101,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
6,Newberg Rock & Dirt,,8790962,000383840,4/10/2015,7/17/2015,104(a),4/10/2015,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
7,Newberg Rock & Dirt,,8790963,000383840,4/10/2015,7/17/2015,104(a),4/13/2015,C,Y,56.15020,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,873.00,Closed,873.00,873.00
8,Newberg Rock & Dirt,,8694458,000345066,1/29/2014,4/12/2014,104(a),1/29/2014,C,N,56.12018,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
9,Newberg Rock & Dirt,,8694457,000345066,1/29/2014,4/12/2014,104(a),1/29/2014,C,N,56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00


In [174]:
df.to_csv("3503598-violations.csv", index=False)

## Put that all in ONE cell that runs correctly

The **entire process**, from searching to saving as a CSV

In [413]:

driver.get("https://arlweb.msha.gov/drs/drshome.htm")
text_input = driver.find_element_by_name('MineId')
driver.execute_script("arguments[0].scrollIntoView(true)", text_input)
text_input.send_keys('3503598')
button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
button.click()

beginning_date = driver.find_element_by_name('BDate')
driver.execute_script("arguments[0].scrollIntoView(true)", beginning_date)
beginning_date.send_keys('1/1/1995')

select_button = driver.find_elements_by_name('Submit')[3]
select_button.send_keys('yes')
select_button.click()

report_button = driver.find_element_by_name('submit')
report_button.click()

reports = driver.find_elements_by_css_selector('tr.drsviols')

my_report = []

for report in reports:
    data = {}
    data['violator'] = report.find_elements_by_css_selector('td.drsviols')[0].text
    data['contract_id'] = " "
    data['citation_no'] = report.find_elements_by_css_selector('td.drsviols')[1].text
    data['case_no'] = report.find_elements_by_css_selector('td.drsviols')[2].text
    data['date_issues'] = report.find_elements_by_css_selector('td.drsviols')[3].text
    data['final_order_date'] = report.find_elements_by_css_selector('td.drsviols')[4].text
    data['section_of_act'] = report.find_elements_by_css_selector('td.drsviols')[5].text
    data['date_terminated'] = report.find_elements_by_css_selector('td.drsviols')[6].text
    data['citation'] = report.find_elements_by_css_selector('td.drsviols')[7].text
    data['s_and_s'] = report.find_elements_by_css_selector('td.drsviols')[8].text
    data['standard'] = report.find_elements_by_css_selector('td.drsviols')[9].text
    
    standard2 = driver.find_elements_by_css_selector('td.drsviols')[9].find_element_by_tag_name('a')
    data['standard_url'] = standard2.get_attribute('href')
    data['proposed_penalty'] = report.find_elements_by_css_selector('td.drsviols')[10].text
    data['citation_status'] = report.find_elements_by_css_selector('td.drsviols')[11].text
    data['current_penalty'] = report.find_elements_by_css_selector('td.drsviols')[12].text
    data['amount_paid'] = report.find_elements_by_css_selector('td.drsviols')[13].text
    my_report.append(data)
    

my_report

import pandas as pd
pd.set_option('display.max_colwidth', -1)
df_one = pd.DataFrame(my_report)
df_one = df[['violator', 'contract_id', 'citation_no', 'case_no', 'date_issues', 'final_order_date', 'section_of_act', 'date_terminated', 'citation', 's_and_s', 'standard', 'standard_url', 'proposed_penalty', 'citation_status', 'current_penalty', 'amount_paid']]
df_one
df_one.to_csv("3503598-violations.csv", index=False)
df_one

,violator,contract_id,citation_no,case_no,date_issues,final_order_date,section_of_act,date_terminated,citation,s_and_s,standard,standard_url,proposed_penalty,citation_status,current_penalty,amount_paid
0,Newberg Rock & Dirt,,8992368,000440809,5/2/2017,7/23/2017,104(a),5/2/2017,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,116.00,Closed,116.00,116.00
1,Newberg Rock & Dirt,,8992214,000435717,2/7/2017,5/21/2017,104(a),2/8/2017,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,116.00,Closed,116.00,116.00
2,Newberg Rock & Dirt,,8872462,000398910,10/21/2015,1/13/2016,104(a),10/27/2015,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
3,Newberg Rock & Dirt,,8872461,000398910,10/21/2015,1/13/2016,104(a),10/27/2015,C,N,56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
4,Newberg Rock & Dirt,,8790965,000383840,4/15/2015,7/17/2015,104(a),4/15/2015,C,N,56.13015,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
5,Newberg Rock & Dirt,,8790964,000383840,4/13/2015,7/17/2015,104(a),4/15/2015,C,N,56.4101,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
6,Newberg Rock & Dirt,,8790962,000383840,4/10/2015,7/17/2015,104(a),4/10/2015,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
7,Newberg Rock & Dirt,,8790963,000383840,4/10/2015,7/17/2015,104(a),4/13/2015,C,Y,56.15020,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,873.00,Closed,873.00,873.00
8,Newberg Rock & Dirt,,8694458,000345066,1/29/2014,4/12/2014,104(a),1/29/2014,C,N,56.12018,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
9,Newberg Rock & Dirt,,8694457,000345066,1/29/2014,4/12/2014,104(a),1/29/2014,C,N,56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00


In [435]:
driver.get("https://arlweb.msha.gov/drs/drshome.htm")
text_input = driver.find_element_by_name('MineId')
driver.execute_script("arguments[0].scrollIntoView(true)", text_input)
text_input.send_keys('0801306')
button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
button.click()

beginning_date = driver.find_element_by_name('BDate')
driver.execute_script("arguments[0].scrollIntoView(true)", beginning_date)
beginning_date.send_keys('1/1/1995')

select_button = driver.find_elements_by_name('Submit')[3]
select_button.send_keys('yes')
select_button.click()

report_button = driver.find_element_by_name('submit')
report_button.click()

reports = driver.find_elements_by_css_selector('tr.drsviols')

my_report = []

for report in reports:
    data = {}
    data['violator'] = report.find_elements_by_css_selector('td.drsviols')[0].text
    data['contract_id'] = " "
    data['citation_no'] = report.find_elements_by_css_selector('td.drsviols')[1].text
    data['case_no'] = report.find_elements_by_css_selector('td.drsviols')[2].text
    data['date_issues'] = report.find_elements_by_css_selector('td.drsviols')[3].text
    data['final_order_date'] = report.find_elements_by_css_selector('td.drsviols')[4].text
    data['section_of_act'] = report.find_elements_by_css_selector('td.drsviols')[5].text
    data['date_terminated'] = report.find_elements_by_css_selector('td.drsviols')[6].text
    data['citation'] = report.find_elements_by_css_selector('td.drsviols')[7].text
    data['s_and_s'] = report.find_elements_by_css_selector('td.drsviols')[8].text
    data['standard'] = report.find_elements_by_css_selector('td.drsviols')[9].text

    try:
        data['standard_url'] = driver.find_elements_by_css_selector('td.drsviols')[9].find_element_by_tag_name('a').get_attribute('href')
    
    except:
        data['standard_url'] = " "

    try:
        data['proposed_penalty'] = report.find_elements_by_css_selector('td.drsviols')[10].text
        data['citation_status'] = report.find_elements_by_css_selector('td.drsviols')[11].text
        data['current_penalty'] = report.find_elements_by_css_selector('td.drsviols')[12].text
        data['amount_paid'] = report.find_elements_by_css_selector('td.drsviols')[13].text
    except:
        data['proposed_penalty'] = " "
        data['citation_status'] = " "
        data['current_penalty'] = " "
        data['amount_paid'] = " "
        
    my_report.append(data)
my_report

import pandas as pd
pd.set_option('display.max_colwidth', -1)
df_2 = pd.DataFrame(my_report)
df_2 = df_2[['violator', 'contract_id', 'citation_no', 'case_no', 'date_issues', 'final_order_date', 'section_of_act', 'date_terminated', 'citation', 's_and_s', 'standard', 'standard_url', 'proposed_penalty', 'citation_status', 'current_penalty', 'amount_paid']]
df_2
df_2.to_csv("0801306-violations.csv", index=False)
df_2

,violator,contract_id,citation_no,case_no,date_issues,final_order_date,section_of_act,date_terminated,citation,s_and_s,standard,standard_url,proposed_penalty,citation_status,current_penalty,amount_paid
0,"Holley Dirt Company, Inc",,9421625,,5/15/2018,104(a),5/15/2018,C,Y,56.20003(a),Not Assessed Yet,,,,,
1,"Holley Dirt Company, Inc",,9335384,000453336,10/26/2017,1/13/2018,104(a),10/26/2017,C,N,47.41(a),,116.00,Closed,116.00,116.00
2,"Holley Dirt Company, Inc",,9335385,000453336,10/26/2017,1/13/2018,104(a),10/26/2017,C,Y,56.9300(a),,116.00,Closed,116.00,116.00
3,"Holley Dirt Company, Inc",,9335386,000453336,10/26/2017,1/13/2018,104(a),10/26/2017,C,N,56.4601,,116.00,Closed,116.00,116.00
4,"Holley Dirt Company, Inc",,8912694,000442250,5/15/2017,8/9/2017,104(a),5/15/2017,C,N,56.14132(a),,116.00,Closed,116.00,116.00
5,"Holley Dirt Company, Inc",,8638781,000427623,11/30/2016,2/18/2017,104(a),12/1/2016,C,N,56.12028,,351.00,Closed,351.00,351.00
6,"Holley Dirt Company, Inc",,8903434,000411633,4/13/2016,7/6/2016,104(a),4/19/2016,C,Y,46.11(d),,117.00,Closed,117.00,117.00
7,"Holley Dirt Company, Inc",,8903433,000411633,4/13/2016,7/6/2016,104(a),4/19/2016,C,Y,56.14100(b),,100.00,Closed,100.00,100.00
8,"Holley Dirt Company, Inc",,8903436,000411633,4/13/2016,7/6/2016,104(a),4/19/2016,C,N,56.12004,,100.00,Closed,100.00,100.00
9,"Holley Dirt Company, Inc",,8903435,000411633,4/13/2016,7/6/2016,104(a),4/13/2016,C,Y,56.9300(a),,117.00,Closed,117.00,117.00


# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [74]:
df_subset = pd.read_csv("mines-subset.csv", dtype='str')
df_subset

,id
0,4104757
1,0801306
2,3609931


### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them? If so, make them match.

- *TIP: You can zero fill if you want, but another option is that when reading in a CSV, `dtype='str'` will force everything to be a string*

### Convert your one-cell scraper into a function, and use it on each row of our dataset

- *TIP: You'll be using `.apply`*
- *TIP: You won't be joining this back into your dataframe, so you don't need to `return` anything or `join` or any of that.*
- *TIP: Be careful of your **other variable names** - if you're calling the thing you're sending your function `row`, you can't use it anywhere else (like in your loop)*
- *TIP: **BE CAREFUL WHAT YOU NAME YOUR DATAFRAMES.** If you name the citations dataframe `df` it can overwrite your mine ID `df`*
- *TIP: You'll be saving a dataframe each time*
- *TIP: Be sure you change everything that refers to the mine ID to refer to the current row's ID instead of `3503598`*
- *TIP: BE SURE TO CHANGE EVERYTHING THAT REFERS TO THE MINE ID*
- *TIP: EVERYTHING, EVERYTHING, EVERYTHING! Look at the end of your function! Maybe I'm overreacting, I don't know.*
- *TIP: If you hit an error about list index out of range, see what line it's happening on and go look at the page. What's different about this page than the previous ones? (answer: the last three columns!) If you assign those columns later using `try`/`except` you should be able to get some data from those rows without throwing it all out. If you can't figure it out, just wrap it all in try/except and give up on those rows*
- *TIP: Some of the standards might not have links, either, so you might want to wrap that in a `try`/`except`, too!*

In [94]:
def get_report_info(row):    
    driver.get("https://arlweb.msha.gov/drs/drshome.htm")
    text_input = driver.find_element_by_name('MineId')
    driver.execute_script("arguments[0].scrollIntoView(true)", text_input)
    text_input.send_keys(row['id'])        
    button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    button.click()
    beginning_date = driver.find_element_by_name('BDate')
    driver.execute_script("arguments[0].scrollIntoView(true)", beginning_date)
    beginning_date.send_keys('1/1/1995')
    select_button = driver.find_elements_by_name('Submit')[3]
    select_button.send_keys('yes')
    select_button.click()
    report_button = driver.find_element_by_name('submit')
    report_button.click()
    reports = driver.find_elements_by_css_selector('tr.drsviols')

    my_report = []

    for report in reports:
        data = {}
        data['violator'] = report.find_elements_by_css_selector('td.drsviols')[0].text
        data['contract_id'] = " "
        data['citation_no'] = report.find_elements_by_css_selector('td.drsviols')[1].text
        data['case_no'] = report.find_elements_by_css_selector('td.drsviols')[2].text
        data['date_issues'] = report.find_elements_by_css_selector('td.drsviols')[3].text
        data['final_order_date'] = report.find_elements_by_css_selector('td.drsviols')[4].text
        data['section_of_act'] = report.find_elements_by_css_selector('td.drsviols')[5].text
        data['date_terminated'] = report.find_elements_by_css_selector('td.drsviols')[6].text
        data['citation'] = report.find_elements_by_css_selector('td.drsviols')[7].text
        data['s_and_s'] = report.find_elements_by_css_selector('td.drsviols')[8].text
        data['standard'] = report.find_elements_by_css_selector('td.drsviols')[9].text
        try:
            standard2 = driver.find_elements_by_css_selector('td.drsviols')[9].find_element_by_tag_name('a')
            data['standard_url'] = standard2.get_attribute('href')
        except:
            data['standard_url'] = " "
        try:
            data['proposed_penalty'] = report.find_elements_by_css_selector('td.drsviols')[10].text
            data['citation_status'] = report.find_elements_by_css_selector('td.drsviols')[11].text
            data['current_penalty'] = report.find_elements_by_css_selector('td.drsviols')[12].text
            data['amount_paid'] = report.find_elements_by_css_selector('td.drsviols')[13].text
        except:
            data['proposed_penalty'] = " "
            data['citation_status'] = " "
            data['current_penalty'] = " "
            data['amount_paid'] = " "        
        my_report.append(data)    
    my_report 
    pd.set_option('display.max_colwidth', -1)
    df_three = pd.DataFrame(my_report)
    df_three = df_three[['violator', 'contract_id', 'citation_no', 'case_no', 'date_issues', 'final_order_date', 'section_of_act', 'date_terminated', 'citation', 's_and_s', 'standard', 'standard_url', 'proposed_penalty', 'citation_status', 'current_penalty', 'amount_paid']]
    df_three.to_csv(row['id'] + "violations.csv",index=False) 

In [92]:
df_subset.apply(get_report_info, axis=1) 

KeyboardInterrupt: 

In [73]:
df_three

,violator,contract_id,citation_no,case_no,date_issues,final_order_date,section_of_act,date_terminated,citation,s_and_s,standard,standard_url,proposed_penalty,citation_status,current_penalty,amount_paid
0,Newberg Rock & Dirt,,8992368,000440809,5/2/2017,7/23/2017,104(a),5/2/2017,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,116.00,Closed,116.00,116.00
1,Newberg Rock & Dirt,,8992214,000435717,2/7/2017,5/21/2017,104(a),2/8/2017,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,116.00,Closed,116.00,116.00
2,Newberg Rock & Dirt,,8872461,000398910,10/21/2015,1/13/2016,104(a),10/27/2015,C,N,56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
3,Newberg Rock & Dirt,,8872462,000398910,10/21/2015,1/13/2016,104(a),10/27/2015,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
4,Newberg Rock & Dirt,,8790965,000383840,4/15/2015,7/17/2015,104(a),4/15/2015,C,N,56.13015,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
5,Newberg Rock & Dirt,,8790964,000383840,4/13/2015,7/17/2015,104(a),4/15/2015,C,N,56.4101,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
6,Newberg Rock & Dirt,,8790962,000383840,4/10/2015,7/17/2015,104(a),4/10/2015,C,N,56.14107(a),http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
7,Newberg Rock & Dirt,,8790963,000383840,4/10/2015,7/17/2015,104(a),4/13/2015,C,Y,56.15020,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,873.00,Closed,873.00,873.00
8,Newberg Rock & Dirt,,8694458,000345066,1/29/2014,4/12/2014,104(a),1/29/2014,C,N,56.12018,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00
9,Newberg Rock & Dirt,,8694457,000345066,1/29/2014,4/12/2014,104(a),1/29/2014,C,N,56.12004,http://www.gpo.gov/fdsys/pkg/CFR-2017-title30-vol1/pdf/CFR-2017-title30-vol1-sec56-14107.pdf,100.00,Closed,100.00,100.00


# Okay, now do it for ALL of the mines

Open up `mines.csv` using pandas and do the same thing, it will just be for more mines this time.

In [84]:
df_mines_more = pd.read_csv("mines.csv", dtype='str')
df_mines_more


,id
0,3503598
1,4801789
2,5001797
3,4608254
4,2103723
5,4104757
6,0801306
7,3901432
8,3609624
9,3609931


In [90]:

def get_info(number):
    
    driver.get("https://arlweb.msha.gov/drs/drshome.htm")
    text_input = driver.find_element_by_name('MineId')
    driver.execute_script("arguments[0].scrollIntoView(true)", text_input)
    text_input.send_keys(number['id'])
        
    button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    button.click()
    beginning_date = driver.find_element_by_name('BDate')
    driver.execute_script("arguments[0].scrollIntoView(true)", beginning_date)
    beginning_date.send_keys('1/1/1995')
    

    select_button = driver.find_elements_by_name('Submit')[3]
    select_button.send_keys('yes')
    select_button.click()
    

    report_button = driver.find_element_by_name('submit')
    report_button.click()

    reports = driver.find_elements_by_css_selector('tr.drsviols')

    my_report = []

    for report in reports:

        info = {}
        info['violator'] = report.find_elements_by_css_selector('td.drsviols')[0].text
        info['contract_id'] = " "
        info['citation_no'] = report.find_elements_by_css_selector('td.drsviols')[1].text
        info['case_no'] = report.find_elements_by_css_selector('td.drsviols')[2].text
        info['date_issues'] = report.find_elements_by_css_selector('td.drsviols')[3].text
        info['final_order_date'] = report.find_elements_by_css_selector('td.drsviols')[4].text
        info['section_of_act'] = report.find_elements_by_css_selector('td.drsviols')[5].text
        info['date_terminated'] = report.find_elements_by_css_selector('td.drsviols')[6].text
        info['citation'] = report.find_elements_by_css_selector('td.drsviols')[7].text
        info['s_and_s'] = report.find_elements_by_css_selector('td.drsviols')[8].text
        info['standard'] = report.find_elements_by_css_selector('td.drsviols')[9].text
  
        try:
            standard2 = driver.find_elements_by_css_selector('td.drsviols')[9].find_element_by_tag_name('a')
            info['standard_url'] = standard2.get_attribute('href')
        except:
            info['standard_url'] = " "

        try:
            info['proposed_penalty'] = report.find_elements_by_css_selector('td.drsviols')[10].text
            info['citation_status'] = report.find_elements_by_css_selector('td.drsviols')[11].text
            info['current_penalty'] = report.find_elements_by_css_selector('td.drsviols')[12].text
            info['amount_paid'] = report.find_elements_by_css_selector('td.drsviols')[13].text
        except:
            info['proposed_penalty'] = " "
            info['citation_status'] = " "
            info['current_penalty'] = " "
            info['amount_paid'] = " "
        
        my_report.append(info)

    
    my_report
    pd.set_option('display.max_colwidth', -1)
    df_report_info = pd.DataFrame(my_report)
    df_report_info = df_report_info[['violator', 'contract_id', 'citation_no', 'case_no', 'date_issues', 'final_order_date', 'section_of_act', 'date_terminated', 'citation', 's_and_s', 'standard', 'standard_url', 'proposed_penalty', 'citation_status', 'current_penalty', 'amount_paid']]
    df_report_info.to_csv(number['id'] + "-violations.csv", index=False)  

In [ ]:
df_subset.apply(get_report_info, axis=1) 